<a href="https://colab.research.google.com/github/Kenneth-Rojas/tareas-ia-kenneth-rojas/blob/main/SpamOrHam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Laboratorio de Redes Neuronales con PLN
El objetivo de este laboratorio es crear un proyecto propio de redes neuronales utilizando técnicas de PLN. Es este caso se utilizará una red profunda con 2 capas ocultas y una capa de salida.

Se realiza una clasificación binaria para clasificar mensajes o correos electrónicos en forma de texto como SPAM o HAM (correos no deseados o correos de interés).

Proyecto realizado por: Kenneth Rojas Rivera

# Librerias de Python requeridas

In [ ]:
!pip install pandas tensorflow scikit-learn imblearn numpy

# Procesamiento de texto
Primero debemos obtener la base de datos con la que entrenaremos al modelo, para este caso se utilizará el dataset sms.csv el cual contiene una etiqueta de spam o ham seguido del texto.

In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Descargar el dataset (formato: "label\tmessage")
url = "https://raw.githubusercontent.com/justmarkham/pycon-2016-tutorial/master/data/sms.tsv"
df = pd.read_csv(url, sep='\t', header=None, names=['label', 'message'])

# Convertimos etiquetas a 0 (ham) y 1 (spam)
df['label_num'] = df.label.map({'ham': 0, 'spam': 1})

X = df['message'].values
y = df['label_num'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



Acto seguido es importante tokenizar y vectorizar los datos que extraidos anteriormente, así como también reordenarlos para evitar una gran densidad datos de spam o ham contiguos que puedan sesgar el modelo.

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.over_sampling import RandomOverSampler

vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)
ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(X_train_vec, y_train)

# Red neuronal profunda
Utilizaremos un modelos de capas secuenciales, la primera y segunda capa corresponden a capas ocultas con activacion ReLU. La primera capa de 64 neuronas clasifica los datos de entrada, la segunda capa de 32 neuronas procesa esta clasificacion y las procesa a la capa de salida.

La capa de salida consta de una única neurona con activacion tipo sigmod para una clasificación binaria efectiva.

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np

model_spam = Sequential()
model_spam.add(Dense(64, activation='relu', input_shape=(X_resampled.shape[1],)))
model_spam.add(Dense(32, activation='relu'))
model_spam.add(Dense(1, activation='sigmoid'))

model_spam.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_spam.fit(X_resampled.toarray(), np.array(y_resampled), epochs=15, batch_size=32, validation_split=0.1)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/15
218/218 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.8259 - loss: 0.4480 - val_accuracy: 0.9987 - val_loss: 0.0182
Epoch 2/15
218/218 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9981 - loss: 0.0121 - val_accuracy: 1.0000 - val_loss: 0.0052
Epoch 3/15
218/218 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.9999 - loss: 0.0027 - val_accuracy: 1.0000 - val_loss: 0.0014
Epoch 4/15
218/218 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9999 - loss: 0.0012 - val_accuracy: 1.0000 - val_loss: 8.2714e-04
Epoch 5/15
218/218 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 1.0000 - loss: 6.1879e-04 - val_accuracy: 1.0000 - val_loss: 6.4785e-04
Epoch 6/15
218/218 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 1.0000 - loss: 3.6744e-04 - val_accuracy: 1.0000 - val_loss: 3.5917e-04
Epoch 7/15
218/218 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 1.0000 - loss: 2.5756e-04 - val_accuracy: 1.0000 - val_loss: 2.6169e-04
Epoch 8/15
218/218 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 1.0000 - lo

# Funcion de prediccion
Definimos la función de llamado a nuestro modelo de red neuronal profunda donde se evalua la probabilidad de que el texto sea SPAM, la función devuelve las etiquetas SPAM para textos con probabilidad alta y HAM para casos de baja probabilidad.

In [20]:
def predecir_spam(texto, modelo):
    vec = vectorizer.transform([texto])
    prob = modelo.predict(vec.toarray())[0][0]
    return 'SPAM' if prob >= 0.5 else 'HAM'


In [19]:
# Ejemplos de prueba de la red profunda

print(predecir_spam("Congratulations! You've won a free ticket to Bahamas!", model_spam))
print(predecir_spam("WINNER!! You have won a $1000 Walmart gift card. Go to http://bit.ly/123456 to claim now!", model_spam))
print(predecir_spam("Hey, are we still meeting at 3pm for coffee?", model_spam))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
SPAM
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
SPAM
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
No SPAM
